In [1]:
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

engine = create_engine('sqlite+pysqlite:///app/db/data.db', echo=True)

In [2]:
from sqlalchemy import select

from app.models.item import Item
from app.models.stock import Stock
from app.models.cat import Cat
from app.dao.db_base import Base

session = Session(engine)

In [3]:
test_stmt = select(Item)

In [4]:
for item in session.scalars(test_stmt):
    print(item.to_dict())

2025-04-16 12:07:35,164 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-16 12:07:35,178 INFO sqlalchemy.engine.Engine SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id 
FROM items
2025-04-16 12:07:35,178 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ()


{'stock_id': 1, 'load_id': None, 'cat_id': 3, 'lot': '00245', 'pallet': '0034', 'roll': '0101', 'note': 'Item note', 'id': 1}
{'stock_id': 2, 'load_id': None, 'cat_id': 3, 'lot': '00245', 'pallet': '0034', 'roll': '0103', 'note': 'Item note', 'id': 2}
{'stock_id': 1, 'load_id': None, 'cat_id': 1, 'lot': '00430', 'pallet': '0055', 'roll': '1022', 'note': 'Item note', 'id': 3}
{'stock_id': 4, 'load_id': None, 'cat_id': 4, 'lot': '20003', 'pallet': '0204', 'roll': '0126', 'note': 'item note', 'id': 4}
{'stock_id': 1, 'load_id': None, 'cat_id': 1, 'lot': '03200', 'pallet': '0304', 'roll': '0023', 'note': 'forn add', 'id': 5}


In [30]:
stmt = (
    select(Item, Stock, Cat)
    .join(Item.stock)
    .join(Item.cat)
    .where(Stock.ready == False)
)

In [31]:
print(stmt)

SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id, stocks.reference, stocks.date, stocks.consignor, stocks.ready, stocks.note AS note_1, stocks.id AS id_1, cats.name, cats.cat, cats.width, cats.weight, cats.note AS note_2, cats.id AS id_2 
FROM items JOIN stocks ON stocks.id = items.stock_id JOIN cats ON cats.id = items.cat_id 
WHERE stocks.ready = false


In [32]:
for item in session.scalars(stmt):
    print(f'{item.to_dict()}')
    print(f'{item.stock.to_dict()}')
    print(f'{item.cat.to_dict()}')

2025-04-16 15:08:56,923 INFO sqlalchemy.engine.Engine SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id, stocks.reference, stocks.date, stocks.consignor, stocks.ready, stocks.note AS note_1, stocks.id AS id_1, cats.name, cats.cat, cats.width, cats.weight, cats.note AS note_2, cats.id AS id_2 
FROM items JOIN stocks ON stocks.id = items.stock_id JOIN cats ON cats.id = items.cat_id 
WHERE stocks.ready = 0
2025-04-16 15:08:56,925 INFO sqlalchemy.engine.Engine [generated in 0.00226s] ()
{'stock_id': 1, 'load_id': None, 'cat_id': 3, 'lot': '00245', 'pallet': '0034', 'roll': '0101', 'note': 'Item note', 'id': 1}
{'reference': 'FEU034/25', 'date': datetime.datetime(2025, 2, 2, 0, 0), 'consignor': 'Pujol', 'ready': False, 'note': 'tech note', 'id': 1}
{'name': 'Visual', 'cat': 'film', 'width': 1800, 'weight': 123.0, 'note': 'Сгенерированное примечание к категории', 'id': 3}
{'stock_id': 2, 'load_id': None, 'cat_id': 3, 'lot': '00245',

In [33]:
result = session.execute(stmt)

2025-04-16 15:48:41,861 INFO sqlalchemy.engine.Engine SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id, stocks.reference, stocks.date, stocks.consignor, stocks.ready, stocks.note AS note_1, stocks.id AS id_1, cats.name, cats.cat, cats.width, cats.weight, cats.note AS note_2, cats.id AS id_2 
FROM items JOIN stocks ON stocks.id = items.stock_id JOIN cats ON cats.id = items.cat_id 
WHERE stocks.ready = 0
2025-04-16 15:48:41,864 INFO sqlalchemy.engine.Engine [cached since 2385s ago] ()


In [39]:
result.all()

[]

In [15]:
stmt = (
    select(Item).
    order_by(Item.id)
)

In [17]:
result = session.execute(stmt)

2025-04-15 15:02:42,269 INFO sqlalchemy.engine.Engine SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id 
FROM items ORDER BY items.id
2025-04-15 15:02:42,272 INFO sqlalchemy.engine.Engine [generated in 0.00242s] ()


In [18]:
result.all()

[(<app.models.item.Item object at 0x7f2b7d4437c0>,),
 (<app.models.item.Item object at 0x7f2b7d442e00>,),
 (<app.models.item.Item object at 0x7f2b7d441870>,),
 (<app.models.item.Item object at 0x7f2b7d441cf0>,),
 (<app.models.item.Item object at 0x7f2b7d442140>,)]

In [19]:
stmt = (
    select(Item, Stock)
    .join(Item.stock)
    .order_by(Stock.date)
)

In [24]:
for row in session.execute(stmt):
    print(f'{row.Stock.reference} {row.Stock.date} {row.Item.pallet}')

2025-04-15 15:08:06,567 INFO sqlalchemy.engine.Engine SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id, stocks.reference, stocks.date, stocks.consignor, stocks.ready, stocks.note AS note_1, stocks.id AS id_1 
FROM items JOIN stocks ON stocks.id = items.stock_id ORDER BY stocks.date
2025-04-15 15:08:06,569 INFO sqlalchemy.engine.Engine [cached since 99.11s ago] ()
FEU034/25 2025-02-02 00:00:00 0034
FEU034/25 2025-02-02 00:00:00 0055
FEU034/25 2025-02-02 00:00:00 0304
FEU036/25 2025-02-10 00:00:00 0034
FEU040/25 2025-02-14 20:53:26.999000 0204


In [5]:
stmt = (
    select(Item, Stock.reference, Stock.date, Cat.name, Cat.cat, Cat.width)
    .join(Item.stock)
    .join(Item.cat)
    .where(Stock.ready == None)
)

print(stmt)

SELECT items.stock_id, items.load_id, items.cat_id, items.lot, items.pallet, items.roll, items.note, items.id, stocks.reference, stocks.date, cats.name, cats.cat, cats.width 
FROM items JOIN stocks ON stocks.id = items.stock_id JOIN cats ON cats.id = items.cat_id 
WHERE stocks.ready IS NULL


In [11]:
for row in session.execute(stmt):
    print(f'{row.id}')
